In [1]:
#scrappers
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import SessionNotCreatedException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions
from selenium.webdriver.chrome.options import Options
from tqdm import tqdm


#other modules
import time, re
import os, pandas as pd, numpy as np

#filter warnings
import warnings
warnings.filterwarnings('ignore')

In [2]:
#driver=webdriver.Chrome('/Users/ERIA/Library/Application Support/WebDriverManager/bin/chromedriver')
driver=webdriver.Chrome('/Users/ERIA/chromedriver')
driver.get('https://www.flashscore.com/')
driver.maximize_window()

In [3]:
#accepting cookies
time.sleep(3)
driver.find_element(By.XPATH,'//*[@id="onetrust-accept-btn-handler"]').click()

In [4]:
#click odds button
driver.find_element(By.XPATH,'//*[@id="live-table"]/div[1]/div[1]/div[3]').click()

In [5]:
#days x_paths list

day_xpaths=['//*[@id="live-table"]/div[1]/div[2]/div/div[4]/div[7]','//*[@id="live-table"]/div[1]/div[2]/div/div[4]/div[6]',
            '//*[@id="live-table"]/div[1]/div[2]/div/div[4]/div[5]','//*[@id="live-table"]/div[1]/div[2]/div/div[4]/div[4]',
            '//*[@id="live-table"]/div[1]/div[2]/div/div[4]/div[3]','//*[@id="live-table"]/div[1]/div[2]/div/div[4]/div[2]',
            '//*[@id="live-table"]/div[1]/div[2]/div/div[4]/div[1]']


In [6]:
#click date picker button
driver.find_element(By.XPATH,'//*[@id="live-table"]/div[1]/div[2]/div/div[2]').click()

#click specific day
driver.find_element(By.XPATH,day_xpaths[0]).click()

In [7]:
#write path (file to store collected data)
write_path='/Users/ERIA/Desktop/last7days.csv'

<h2>ODDS</h2>

In [8]:
time.sleep(5)

In [9]:
%%time
#home odds
home_odd=[]

h_odd_ob=driver.find_elements(By.CSS_SELECTOR,'div.odds__odd.event__odd--odd1.win.icon.icon--arrow')
for ho in tqdm(h_odd_ob):
    home_odd.append(ho.text)
print(len(home_odd))
print(home_odd[:7])

100%|██████████| 314/314 [00:06<00:00, 50.78it/s]

314
['2.01', '2.71', '3.16', '2.20', '3.86', '1.81', '2.09']
CPU times: user 470 ms, sys: 84.4 ms, total: 555 ms
Wall time: 7.95 s


In [10]:

#draw odds
draw_odd=[]

d_odd_ob=driver.find_elements(By.CSS_SELECTOR,'div.odds__odd.event__odd--odd2.icon.icon--arrow')
for do in tqdm(d_odd_ob):
    try:
        draw_odd.append(do.text)
    except:
        continue
print(len(draw_odd))
print(draw_odd[:10])

100%|██████████| 1013/1013 [00:27<00:00, 36.84it/s]

1013
['3.56', '3.60', '3.85', '3.12', '3.46', '3.47', '3.54', '3.98', '3.70', '3.44']


In [11]:

#away odds
away_odd=[]
a_odd_ob=driver.find_elements(By.CSS_SELECTOR,'div.odds__odd.event__odd--odd3.icon.icon--arrow')
for ao in tqdm(a_odd_ob):
    away_odd.append(ao.text)
print(len(away_odd))
print(away_odd[:7])

100%|██████████| 1013/1013 [00:47<00:00, 21.42it/s]

1013
['3.95', '2.71', '1.94', '3.05', '2.93', '2.35', '2.19']


We'll keep just the first 314 odds for draw and away data using [:314] since it's what has been collected for home. In otherwords to have the same number of records in each list

In [12]:
#odds_df=pd.DataFrame({'home':home_odd[:300],'draw':draw_odd[:300],'away':away_odd})
odds_df=pd.DataFrame({'home':home_odd,'draw':draw_odd[:314],'away':away_odd[:314]})
odds_df

,home,draw,away
0,2.01,3.56,3.95
1,2.71,3.60,2.71
2,3.16,3.85,1.94
3,2.20,3.12,3.05
4,3.86,3.46,2.93
...,...,...,...
309,1.82,4.10,1.66
310,1.64,-,-
311,1.46,3.38,4.94
312,1.64,3.51,4.34


<h2>RESULTS</h2>

In [13]:
#results
home_scores_obj=driver.find_elements(By.CSS_SELECTOR,'div.event__score.event__score--home')
away_scores_obj=driver.find_elements(By.CSS_SELECTOR,'div.event__score.event__score--away')
home_scores=[]
away_scores=[]

In [14]:
for h in tqdm(home_scores_obj):
    home_scores.append(h.text)
print(len(home_scores))
home_scores[:7]

100%|██████████| 996/996 [00:39<00:00, 24.94it/s]

996


['5', '3', '1', '0', '1', '2', '1']

In [15]:
for a in tqdm(away_scores_obj):
    away_scores.append(a.text)
print(len(away_scores))
print(away_scores[:7])

100%|██████████| 996/996 [00:28<00:00, 34.84it/s]

996
['2', '1', '3', '1', '1', '0', '1']


In [16]:
results_df=pd.DataFrame({'h':home_scores, 'a':away_scores})
results_df.shape

(996, 2)

In [17]:
df=pd.concat([odds_df,results_df],axis=1)
df.head()

,home,draw,away,h,a
0,2.01,3.56,3.95,5,2
1,2.71,3.60,2.71,3,1
2,3.16,3.85,1.94,1,3
3,2.20,3.12,3.05,0,1
4,3.86,3.46,2.93,1,1


In [18]:
df.shape

(996, 5)

In [19]:
df.to_csv(write_path)